In [1]:
import pandas as pd
import os

def convert_mutation_format(input_file, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Read the original file with correct column names
    columns = ['Project', 'Sample', 'ID', 'Genome', 'mut_type', 
              'chrom', 'chrom2', 'pos_end', 'ref', 'alt', 'Type']
    
    df = pd.read_csv(input_file, 
                     sep='\t',
                     names=columns,
                     skiprows=1)
    
    # Create new dataframe with required columns
    df_formatted = pd.DataFrame({
        'chr': df['chrom'],
        'pos': df['pos_end'],
        'id': df['Sample'],
        'ref': df['ref'],
        'alt': df['alt']
    })
    
    # Group by sample ID and save separate files
    for sample_id, sample_data in df_formatted.groupby('id'):
        # Create output filename
        output_file = os.path.join(output_dir, f"{sample_id}.vcf")
        
        # Sort by chromosome and position
        sample_data_sorted = sample_data.sort_values(['chr', 'pos'])
        
        # Save to file
        sample_data_sorted[['chr', 'pos', 'id', 'ref', 'alt']].to_csv(
            output_file, 
            sep='\t', 
            index=False, 
            header=False
        )
        
        print(f"Created VCF file for sample {sample_id}: {output_file}")

# Usage example:
convert_mutation_format('560.txt', '560')


Created VCF file for sample PD10010a: 560/PD10010a.vcf
Created VCF file for sample PD10011a: 560/PD10011a.vcf
Created VCF file for sample PD10014a: 560/PD10014a.vcf
Created VCF file for sample PD11326a: 560/PD11326a.vcf
Created VCF file for sample PD11327a: 560/PD11327a.vcf
Created VCF file for sample PD11336a: 560/PD11336a.vcf
Created VCF file for sample PD11337a: 560/PD11337a.vcf
Created VCF file for sample PD11338a: 560/PD11338a.vcf
Created VCF file for sample PD11339a: 560/PD11339a.vcf
Created VCF file for sample PD11340a: 560/PD11340a.vcf
Created VCF file for sample PD11341a: 560/PD11341a.vcf
Created VCF file for sample PD11342a: 560/PD11342a.vcf
Created VCF file for sample PD11343a: 560/PD11343a.vcf
Created VCF file for sample PD11344a: 560/PD11344a.vcf
Created VCF file for sample PD11345a: 560/PD11345a.vcf
Created VCF file for sample PD11346a: 560/PD11346a.vcf
Created VCF file for sample PD11347a: 560/PD11347a.vcf
Created VCF file for sample PD11348a: 560/PD11348a.vcf
Created VC

In [5]:
import os

def create_vcf_report(directory):
    # Initialize empty list to store results
    report_lines = []
    total_variants = 0
    
    # Get all vcf files in directory
    vcf_files = [f for f in os.listdir(directory) if f.endswith('.vcf')]
    
    # Count lines for each vcf
    for vcf_file in sorted(vcf_files):  # sorted for consistent output
        file_path = os.path.join(directory, vcf_file)
        with open(file_path, 'r') as f:
            line_count = sum(1 for line in f)
            
        # Store the results
        sample_name = vcf_file.replace('.vcf', '')
        report_lines.append(f"{sample_name}\t{line_count}")
        total_variants += line_count
    
    # Create the report
    report_path = os.path.join(directory, 'vcf_line_counts.txt')
    with open(report_path, 'w') as f:
        # Write header
        f.write("Sample\tVariant_Count\n")
        
        # Write counts for each sample
        for line in report_lines:
            f.write(line + '\n')
            
        # Write total
        f.write(f"\nTotal variants: {total_variants}")
    
    print(f"Report created at: {report_path}")
    return report_path

# Usage example:
create_vcf_report('by_chr')


Report created at: by_chr/vcf_line_counts.txt


'by_chr/vcf_line_counts.txt'

In [4]:
import os
import shutil

def distribute_vcfs_with_largest_group(vcf_directory, files_per_group=80, num_groups=7):
    # Get all VCF files and their sizes
    file_sizes = []
    for vcf_file in os.listdir(vcf_directory):
        if vcf_file.endswith('.vcf'):
            file_path = os.path.join(vcf_directory, vcf_file)
            with open(file_path, 'r') as f:
                line_count = sum(1 for _ in f)
            file_sizes.append((vcf_file, line_count))
    
    # Sort files by size (largest to smallest)
    file_sizes.sort(key=lambda x: x[1], reverse=True)
    
    # Create groups
    groups = [[] for _ in range(num_groups)]
    
    # Put the largest 80 files in group 0
    groups[0] = file_sizes[:files_per_group]
    
    # Distribute remaining files evenly among the other groups
    remaining_files = file_sizes[files_per_group:]
    remaining_files.sort(key=lambda x: x[1])  # Sort smaller files (smallest first)
    
    for i, file_info in enumerate(remaining_files):
        group_idx = (i % (num_groups - 1)) + 1  # Start from group 1 (index 1)
        groups[group_idx].append(file_info)
    
    # Create group folders and move files
    for i, group in enumerate(groups):
        # Create group folder
        group_dir = os.path.join(vcf_directory, f'group_{i}')
        os.makedirs(group_dir, exist_ok=True)
        
        # Create report
        report_path = os.path.join(group_dir, f'group_{i}_report.txt')
        with open(report_path, 'w') as f:
            f.write(f"Files in group {i}:\n")
            f.write("Filename\tVariants\n")
            total_variants = 0
            
            # Sort files by variant count (ascending)
            for vcf_file, line_count in sorted(group, key=lambda x: x[1]):
                # Move file
                src = os.path.join(vcf_directory, vcf_file)
                dst = os.path.join(group_dir, vcf_file)
                shutil.move(src, dst)
                
                # Write to report
                f.write(f"{vcf_file}\t{line_count}\n")
                total_variants += line_count
            
            # Add summary
            f.write(f"\nSummary for group {i}:\n")
            f.write(f"Total files: {len(group)}\n")
            f.write(f"Total variants: {total_variants}\n")
            avg_variants = total_variants/len(group) if group else 0
            f.write(f"Average variants per file: {avg_variants:.2f}\n")
    
    # Create main summary
    with open(os.path.join(vcf_directory, 'groups_summary.txt'), 'w') as f:
        f.write("Groups Summary:\n")
        f.write("==============\n")
        f.write("Group\tFiles\tTotal_Variants\tAvg_Variants\n")
        
        for i, group in enumerate(groups):
            total = sum(size for _, size in group)
            avg = total/len(group) if group else 0
            f.write(f"Group_{i}\t{len(group)}\t{total}\t{avg:.2f}\n")
            
        # Add note about special distribution
        f.write("\nNote: Group_0 contains the 80 largest files. The remaining files are distributed evenly among the other groups.")
    
    print(f"Created {num_groups} groups. Group 0 contains the 80 largest files, the rest are distributed evenly.")
    return groups

# Usage
distribute_vcfs_with_largest_group('output_directory')


Created 7 groups. Group 0 contains the 80 largest files, the rest are distributed evenly.


[[('PD13604a.vcf', 93102),
  ('PD23579a.vcf', 87311),
  ('PD23564a.vcf', 83447),
  ('PD5937a.vcf', 76097),
  ('PD23561a.vcf', 58028),
  ('PD9063a.vcf', 50045),
  ('PD4977a.vcf', 42258),
  ('PD6405a.vcf', 35317),
  ('PD13425a.vcf', 34502),
  ('PD9568a.vcf', 30395),
  ('PD24189a.vcf', 29590),
  ('PD4072a.vcf', 27931),
  ('PD8832a.vcf', 26409),
  ('PD8660a2.vcf', 26128),
  ('PD4607a.vcf', 25953),
  ('PD24208a.vcf', 25624),
  ('PD6412a.vcf', 25187),
  ('PD24326a.vcf', 23620),
  ('PD24333a.vcf', 22884),
  ('PD6043a.vcf', 22764),
  ('PD11465a.vcf', 22551),
  ('PD8982a.vcf', 21557),
  ('PD24201a.vcf', 20478),
  ('PD24197a.vcf', 20215),
  ('PD18045a.vcf', 19468),
  ('PD11379a.vcf', 19382),
  ('PD13296a.vcf', 18582),
  ('PD7426a.vcf', 18497),
  ('PD9576a.vcf', 17838),
  ('PD24192a.vcf', 17827),
  ('PD18020a.vcf', 17772),
  ('PD11751a.vcf', 16648),
  ('PD24327a.vcf', 16185),
  ('PD7428a.vcf', 15950),
  ('PD8652a2.vcf', 15856),
  ('PD24320a.vcf', 15695),
  ('PD8980a.vcf', 15459),
  ('PD24217a.vcf